# Atmospheric Correction (example 1)
*---*

### THE PROBLEM

A common problem in remote sensing is the need to to convert at-sensor radiance (L) to surface reflectance (ρ). One way to describe the relationship between the two is:

ρ = π(L -Lp) / τ(Edir + Edif)

where

* ρ = surface reflectance
* π = 3.1415..
* L = at-sensor radiance
* Lp = path radiance
* τ = transmissivity (from surface to satellite)
* Edir = direct solar irradiance 
* Edif = diffuse solar irradiance



Let's say we know the at-sensor radiance

In [78]:
L = 100

We now have 2 knowns in this equation (i.e. π and L) and need to obtain the remaining 4 unknowns (i.e. Edir, Edif, τ and Lp) to calculate surface reflectance. 

### THE SOLUTION

The 4 unknowns depend on Earth-Sun-Satellite geometry and atmospheric conditions

**Earth-Sun-Satellite Geometry** <br />
This can be defined using the following variables

* solar zenith angle (degrees)
* view zentith angle (degrees)
* Earth-Sun distance (Astronomical Units)
* target altitude (km)

This information is often provided in the metadata of satellite imagery

In [79]:
solar_z = 20
view_z = 0   
ESd = 1
alt = 0

**Atmospheric Conditions** <br />
For visible, near-infrared and short-wave infrared wavelengths the key variables are:

* water vapour column
* ozone
* aerosol optical thickness

We can obtain values for the water vapour column from [NCEP/NCAR](http://journals.ametsoc.org/doi/abs/10.1175/1520-0477%281996%29077%3C0437%3ATNYRP%3E2.0.CO%3B2) and for ozone from [TOMS/OMI](http://ozoneaq.gsfc.nasa.gov/missions). Aerosol optical thickness, however, is a bit more tricky, we could use the [MODIS Aerosol Product](http://modis-atmos.gsfc.nasa.gov/MOD04_L2/index.html) or try and estimate it using an in-scene technique.

Let's say we have the water vapour column (H2O), ozone (O3) and aerosol optical thickness (AOT)

In [80]:
H2O = 1
O3 = 0.4
AOT = 0.3

**How do we use Atmcorr to convert at-sensor radiance to surface reflectance?**

First we need to open up the relevant interpolated lookup table (iLUT). An iLUT is like a simulation of radiative transfer (RT) code but it runs orders of magnitude faster. Please see our [validation](https://github.com/samsammurphy/atmcorr/issues/7) page. There is more information on how iLUTs are [organized](https://github.com/samsammurphy/atmcorr/issues/8) as well.


Let's say we want to correct the blue band (i.e. B2) of Landsat 8's OLI sensor using a continental (CO) aerosol model

In [81]:
import pickle #i.e. iLUTs are currently saved as Python pickle files

iLUT_path = '/home/sam/git/atmcorr/iLUTs/LANDSAT_OLI_MA/viewz_0/LANDSAT_OLI_MA_0_B2.ilut'
with open(iLUT_path, 'rb') as f:
    iLUT = pickle.load(f)

Then we can use the iLUT to retrieve the unknowns:

In [82]:
Edir, Edif, tau2, Lp = iLUT(solar_z,H2O,O3,AOT,alt)

and get the surface reflectance:

In [87]:
import math
ρ = (math.pi*(L-Lp))/(tau2*(Edir+Edif))

print('Surface Reflectance = {}'.format(ρ))

Surface Reflectance = 0.09881279422575164


# (need to explain the orbital correction..)